In [ ]:
!nvidia-smi

Thu Nov 20 01:11:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt-get update -y
!apt-get install -y cuda-toolkit-12-4     # we install this toolkit, so that nvcc version is same as the CUDA driver version (seen in nvidia-smi as CUDA Version: 12.4)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,830 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy In

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%writefile vectorAdd.cu
// VECTOR ADDITION

#include <stdio.h>

#define cudaCheckError(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__global__ void vectorAdd(float *A, float *B, float *C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;  // Get index accessed by thread using threadId and blockId
    if (i < N)
        C[i] = A[i] + B[i];
}

int main() {
    int N = 1 << 20;
    size_t size = N * sizeof(float);

    float *A, *B, *C;
    cudaCheckError(cudaMallocManaged(&A, size));    // cudaMallocManaged creates common memory,accessible from CPU and GPU both
    cudaCheckError(cudaMallocManaged(&B, size));
    cudaCheckError(cudaMallocManaged(&C, size));

    for (int i = 0; i < N; i++) { A[i] = i; B[i] = 2*i; }

    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    vectorAdd<<<blocks, threads>>>(A, B, C, N);
    cudaCheckError(cudaGetLastError());
    cudaCheckError(cudaDeviceSynchronize());

    printf("A[100]=%f, B[100]=%f, C[100]=%f\n", A[100], B[100], C[100]);

    cudaFree(A); cudaFree(B); cudaFree(C);
    return 0;
}


Overwriting vectorAdd.cu


In [ ]:
!nvcc -arch=sm_75  vectorAdd.cu -o vector_add # Add -arch=sm_75 for the T4 GPU
!./vector_add

A[100]=100.000000, B[100]=200.000000, C[100]=300.000000


In [ ]:
%%writefile matMul.cu
// MATRIX MULTIPLICATION

#include <stdio.h>
#define N 16

__global__ void matMul(float *A, float *B, float *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < n && col < n) {
        float sum = 0;
        for (int k = 0; k < n; k++)
            sum += A[row*n + k] * B[k*n + col];
        C[row*n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);
    float *A, *B, *C;

    // cudaMallocManaged allocates the memory on both device and host
    cudaMallocManaged(&A, size);
    cudaMallocManaged(&B, size);
    cudaMallocManaged(&C, size);

    for (int i = 0; i < N*N; i++) { A[i] = 1.0f; B[i] = 2.0f; }

    // So, each block has threads that are composed of 16 threads along x-axis and 16 threads along y-axis
    // This is small enough to fit in shared memory (all threads in block share the 'shared memory')
    dim3 threads(16, 16);

    // number of blocks = (N+15)/16 X (N+15)/16
    dim3 blocks((N + 15)/16, (N + 15)/16);

    // Each thread computes one output element of matrix C; C and N are arguments
    matMul<<<blocks, threads>>>(A, B, C, N);
    cudaDeviceSynchronize();

    printf("C[0] = %f\n", C[0]);
    cudaFree(A); cudaFree(B); cudaFree(C);
    return 0;
}


Writing matMul.cu


In [ ]:
!nvcc -arch=sm_75  matMul.cu -o matMul # Add -arch=sm_75 for the T4 GPU
!./matMul

C[0] = 32.000000


In [ ]:
%%writefile reduceSum.cu
// VECTOR SUM
#include <stdio.h>

__global__ void reduceSum(float *input, float *output, int N) {
   // This is way to initialize the dynamic shared memory array, size of which has been declared during kernel launch
    extern __shared__ float sdata[];
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // such an i<N check will almost always be required, as our total number of threads (blocks*threads) will cross N slightly most
    // of the times as we did blocks = (N+threads-1)/threads
    sdata[tid] = (i < N) ? input[i] : 0.0f;
    // all threads within block stopped till everyone comes (barrier synchronization)
    __syncthreads();

    // So sdata[] is from sdata[0] to sdata[255]
    // Note that halving threads each time and adding like this only works if number of threads in block is a power of 2
    // Even for last block when i>=N is possible, sdata[] is still of size 255 and for i>=N, sdata[tid]=0.0f
    for (int stride = blockDim.x/2; stride > 0; stride >>= 1) {
        if (tid < stride)
            sdata[tid] += sdata[tid + stride];
        __syncthreads();
    }

    if (tid == 0)
        output[blockIdx.x] = sdata[0];
}

int main() {
    int N = 1024;
    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    float *input, *partial, *result;
    cudaMallocManaged(&input, N * sizeof(float));
    cudaMallocManaged(&partial, blocks * sizeof(float));
    cudaMallocManaged(&result, sizeof(float));

    for (int i = 0; i < N; i++) input[i] = 1.0f;

    // kernelName<<<gridDim, blockDim, sharedMemSize, stream>>>(args...);
    // sharedMemSize = size of shared memory in bytes; shared memory is dynamic memory created within block
    // Each Streaming Multiprocessor (SM) has a small pool of on-chip shared memory (like an L1 scratchpad) that is divided among concurrently running blocks.
    // T4 GPU has 48 KB shared memory per block
    reduceSum<<<blocks, threads, threads*sizeof(float)>>>(input, partial, N);
    cudaDeviceSynchronize();

    *result = 0;
    for (int i = 0; i < blocks; i++) *result += partial[i];

    printf("Sum = %f\n", *result);
    cudaFree(input); cudaFree(partial); cudaFree(result);
    return 0;
}


Writing reduceSum.cu


In [ ]:
!nvcc -arch=sm_75  reduceSum.cu -o reduceSum # Add -arch=sm_75 for the T4 GPU
!./reduceSum

Sum = 1024.000000


In [ ]:
%%writefile simple_pointwise_mult.cu
// POINTWISE MODULAR MULTIPLICATION - 100X faster than sequential multiplication of coefficients

#include <stdint.h>
#include <stdio.h>
#include <chrono>

__global__ void pointwise_mul_u128(const uint64_t* A,
                                   const uint64_t* B,
                                   uint64_t* C,
                                   uint64_t q,
                                   size_t N) {
    size_t i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= N) return;
    // product of 64-bit integers will give 128-bit integer
    unsigned __int128 prod = (unsigned __int128)A[i] * (unsigned __int128)B[i];
    /*if(i==100)
    {
      printf("i: %lu\n",i);
      printf("a: %lu, b: %lu\n", A[i],B[i]);
    }
    */

    // This modulus is in fact expensive, due to which Montgomery multiplication is used which just uses bit shifts, etc.
    uint64_t r = (uint64_t)(prod % q);
    C[i] = r;
}

int main() {
    int N = 1 << 20;
    size_t size = N * sizeof(uint64_t);

    uint64_t *A, *B, *C;
    cudaMallocManaged(&A, size);
    cudaMallocManaged(&B, size);
    cudaMallocManaged(&C, size);

    for (int i = 0; i < N; i++) { A[i] = i; B[i] = 2*i; }

    A[100]=(1ULL<<50)-1;
    B[100]=(1ULL<<59)+3;

    printf("A[100] = %llu\n", (unsigned long long)A[100]);
    printf("B[100] = %llu\n", (unsigned long long)B[100]);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    // large prime
    uint64_t q = (1ULL << 61) - 1;

    auto start = std::chrono::high_resolution_clock::now();
    pointwise_mul_u128<<<blocks, threads>>>(A, B, C, q, N);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> elapsed = end - start;

    cudaDeviceSynchronize();

    printf("A[100]=%lu, B[100]=%lu, C[100]=%lu\n", A[100], B[100], C[100]);
    printf("CPU execution time: %.3f ms\n", elapsed.count());

    cudaFree(A); cudaFree(B); cudaFree(C);
    return 0;
}

Writing simple_pointwise_mult.cu


In [ ]:
!nvcc -arch=sm_75  simple_pointwise_mult.cu -o simple_pointwise_mult # Add -arch=sm_75 for the T4 GPU
!time ./simple_pointwise_mult

A[100] = 1125899906842623
B[100] = 576460752303423491
A[100]=1125899906842623, B[100]=576460752303423491, C[100]=1733041431607508988
CPU execution time: 0.166 ms

real	0m0.146s
user	0m0.024s
sys	0m0.118s


In [ ]:
%%writefile simple_pointwise_mult_cpu.cpp
#include <stdint.h>
#include <stdio.h>
#include <stdlib.h>
#include <chrono>

int main() {
    int N = 1 << 20;  // number of coefficients
    uint64_t q = (1ULL << 61) - 1;  // 2^61 - 1 (Mersenne prime)

    uint64_t *A = (uint64_t*)malloc(N * sizeof(uint64_t));
    uint64_t *B = (uint64_t*)malloc(N * sizeof(uint64_t));
    uint64_t *C = (uint64_t*)malloc(N * sizeof(uint64_t));

    for (int i = 0; i < N; i++) {
        A[i] = i;
        B[i] = 2 * i;
    }

    // Use large test values at index 100
    A[100] = (1ULL << 50) - 1;
    B[100] = (1ULL << 59) + 3;


    auto start = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < N; i++) {
        __uint128_t prod = (__uint128_t)A[i] * (__uint128_t)B[i];
        C[i] = (uint64_t)(prod % q);

        // if (i == 100) {
        //     printf("i = %d\n", i);
        //     printf("A[100] = %llu\n", (unsigned long long)A[i]);
        //     printf("B[100] = %llu\n", (unsigned long long)B[i]);
        //     printf("prod = %llu (low 64 bits)\n", (unsigned long long)prod);
        //     printf("C[100] = %llu\n", (unsigned long long)C[i]);
        // }
    }
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> elapsed = end - start;
    printf("CPU execution time: %.3f ms\n", elapsed.count());

    printf("\nHost summary:\nA[100]=%llu, B[100]=%llu, C[100]=%llu\n",
           (unsigned long long)A[100],
           (unsigned long long)B[100],
           (unsigned long long)C[100]);

    free(A);
    free(B);
    free(C);
    return 0;
}


Writing simple_pointwise_mult_cpu.cpp


In [ ]:
# Pointwise Modular multiplication
!g++ -std=c++17 simple_pointwise_mult_cpu.cpp -o simple_pointwise_mult_cpu
!./simple_pointwise_mult_cpu

CPU execution time: 17.595 ms

Host summary:
A[100]=1125899906842623, B[100]=576460752303423491, C[100]=1733041431607508988


In [ ]:
%%writefile montmogery_pointwise_mult.cu

#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <chrono>

__global__ void montgomery_mul_u128(const uint64_t* A,
                                   const uint64_t* B,
                                   uint64_t* C,
                                   uint64_t q,
                                   size_t N, uint64_t qinv_mod_R, int k)
{
    size_t i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i>=N)
      return;

    uint64_t a=A[i]; uint64_t b=B[i];

    __uint128_t t = (__uint128_t)a * (__uint128_t)b;
    // uint64_t Rmask = 0xFFFFFFFFFFFFFFFFULL;
    // '& Rmask' ensures that we take only the bottom 64 bits
    // uint64_t m = (uint64_t)((__uint128_t)(t * qinv_mod_R) & Rmask);  // (t * qinv) mod R
    uint64_t m = (uint64_t)((__uint128_t)t * qinv_mod_R);   // automatically truncates to lower 64 bits; no need to do '& Rmask' explicitly
    __uint128_t temp = (__uint128_t)(t + (__uint128_t)m * (__uint128_t)q);
    uint64_t u = (uint64_t)(temp >> k); // divide by R (shift);
    if (u >= q) u -= q;

    /*
    a'<q, b'<q
    q<R
    so, a'*b' = t <qR
    m<R, so m*q < R*q
    So, (t+m*q) < 2*q*R
    So, (t+m*q)/R < 2*q
    */
    C[i]=u;

}



int main() {

    int N = 1 << 20;
    size_t size = N * sizeof(uint64_t);

    uint64_t *A, *B, *C, *C1;
    cudaMallocManaged(&A, size);
    cudaMallocManaged(&B, size);
    cudaMallocManaged(&C, size);
    cudaMallocManaged(&C1, size);

    uint64_t *A1, *B1;
    A1 = (uint64_t*)malloc(N * sizeof(uint64_t));
    B1 = (uint64_t*)malloc(N * sizeof(uint64_t));

    for (int i = 0; i < N; i++) { A1[i] = i; B1[i] = 2*i; }

    A1[100]=(1ULL<<50)-1;
    B1[100]=(1ULL<<59)+3;

    printf("A1[3] = %llu\n", (unsigned long long)A1[3]);
    printf("B1[3] = %llu\n", (unsigned long long)B1[3]);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    // large prime
    uint64_t q = (1ULL << 61) - 1;
    uint64_t qinv_mod_R = 2305843009213693953;

    int k=64;
    __uint128_t R=((__uint128_t)1<<k);

    for(int i=0;i<N;i++)
    {
      __uint128_t x = __uint128_t(A1[i])*R;
      A[i]=(uint64_t)(x%q);
      if(i==3)
        printf("A[%d]=%llu\n",i,(unsigned long long)A[i]);
    }

    for(int i=0;i<N;i++)
    {
      __uint128_t x = __uint128_t(B1[i])*R;
      B[i]=(uint64_t)(x%q);
    }


    auto start = std::chrono::high_resolution_clock::now();
    montgomery_mul_u128<<<blocks, threads>>>(A, B, C, q, N, qinv_mod_R, k);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> elapsed = end - start;

    cudaDeviceSynchronize();

    for(int i=0;i<N;i++)
    {
      B[i]=1;
    }

    montgomery_mul_u128<<<blocks,threads>>>(C,B,C1,q,N,qinv_mod_R,k);
    cudaDeviceSynchronize();

    printf("A1[3]=%lu, B1[3]=%lu, C[3]: %lu, B[3]: %lu, C1[3]=%lu\n", A1[3], B1[3], C[3], B[3], C1[3]);
    printf("A1[100]=%lu, B1[100]=%lu, C[100]: %lu, B[100]: %lu, C1[100]=%lu\n", A1[100], B1[100], C[100], B[100], C1[100]);
    printf("Execution time: %.3f ms\n", elapsed.count());

    cudaFree(A); cudaFree(B); cudaFree(C); cudaFree(C1);
    free(A1); free(B1);

    return 0;
}


Overwriting montmogery_pointwise_mult.cu


In [ ]:
!nvcc -arch=sm_75  montmogery_pointwise_mult.cu -o montmogery_pointwise_mult # Add -arch=sm_75 for the T4 GPU
!time ./montmogery_pointwise_mult

# Note that my Montomgery GPU kernel code is slower than normal GPU kernel code that used %.
# This is because modulo in GPU is translated to Barrett-like optimization by compiler internally.

A1[3] = 3
B1[3] = 6
A[3]=24
A1[3]=3, B1[3]=6, C[3]: 144, B[3]: 1, C1[3]=18
A1[100]=1125899906842623, B1[100]=576460752303423491, C[100]: 29273397577908198, B[100]: 1, C1[100]=1733041431607508988
Execution time: 0.172 ms

real	0m0.243s
user	0m0.115s
sys	0m0.124s


In [ ]:

# Barrett reduction -
# x mod m
# r = x - floor(x * μ / 2ⁿ) * m
# x mod m = (r>=q? r-q:r)
# where μ = floor(2ⁿ / m)

# Montmogery reduction:
# Let R = 2⁶⁴ (or 2¹²⁸) and precompute m′ = -(m⁻¹ mod R)
# Reduction:
# t = x * m′ mod R
# u = (x + t*m) / R
# if u ≥ m: u -= m
# return u

# Barrett Used When:
# You need to reduce arbitrary numbers modulo m
# Modulus changes often
# NTT / RNS arithmetic (as in SEAL)
# GPU kernels (no domain change cost)
# Polynomial operations

# Montgomery Used When:
# You perform thousands/millions of modular multiplications under the same modulus
# Cryptosystems:
# RSA
# ECC
# Pairings
# Homomorphic crypto in large integer arithmetic

# Montgomery is the standard for CPU-side big integer crypto.



In [ ]:
/*
Barrett reduction implementation

Barrett reduction -
x mod m:
r = x - floor(x * μ / 2ⁿ) * m
x mod m = (r>=q? r-q:r)
where μ = floor(2ⁿ / m)

*/

__device__ inline uint64_t barrett_reduce_64_kernel(uint64_t input, uint64_t modulus_value, uint64_t ratio)
  {
      // Reduces input using base 2^64 Barrett reduction
      // floor(2^64 / mod) == floor( floor(2^128 / mod) )

      unsigned long long tmp[2];

      auto input_original = input;

      multiply_uint64_hw64_kernel(input, ratio, tmp + 1);

      tmp[0] = input_original - tmp[1] * modulus_value;
      return static_cast<uint64_t>(tmp[0] >= modulus_value ? tmp[0] - modulus_value : tmp[0]);
  }


In [ ]:
%%writefile multiply64.cu
/*
operand1 = high1 | low1
operand2 = high2 | low2

where each is a 32-bit part
low1  = operand1 & 0xFFFFFFFF
high1 = operand1 >> 32
low2  = operand2 & 0xFFFFFFFF
high2 = operand2 >> 32

School-book multiplication -
operand1 * operand2 =

(high1 * high2 << 64)
+ (high1 * low2 << 32)
+ (high2 * low1 << 32)
+ (low1 * low2)
*/

inline void multiply_u64_generic(uint64_t a, uint64_t b, uint64_t *hi, uint64_t *lo)
{
    const uint64_t MASK32 = 0xFFFFFFFFULL;

    uint64_t a_lo = a & MASK32;
    uint64_t b_lo = b & MASK32;
    uint64_t a_hi = a >> 32;
    uint64_t b_hi = b >> 32;

    /* compute low part = a_lo * b_lo */
    uint64_t low_low_lo = (uint64_t)(a_lo * b_lo);             // low 32 bits and high 32 bits inside
    uint64_t low_low_hi = (uint64_t)(((__uint128_t)a_lo * b_lo) >> 32); // we can use __uint128_t for this small step if available, but to keep pure C we also do it below

    /* compute cross terms */
    uint64_t cross1 = a_hi * b_lo;  // 64-bit
    uint64_t cross2 = a_lo * b_hi;  // 64-bit

    /* sum the low 32-bits region: (low_low_hi) + (cross1 & MASK32) + (cross2 & MASK32) */
    uint64_t t0_low32 = low_low_hi + (cross1 & MASK32) + (cross2 & MASK32);
    uint64_t t0_low32_lo = t0_low32 & MASK32;
    uint64_t carry_t0 = t0_low32 >> 32;

    /* compute high part: a_hi*b_hi + (cross1 >> 32) + (cross2 >> 32) + carry_t0 */
    uint64_t t1 = a_hi * b_hi;
    t1 += (cross1 >> 32);
    t1 += (cross2 >> 32);
    t1 += carry_t0;

    /* Compose final 128-bit result:
       low 64 = (t0_low32_lo << 32) | (low_low_lo & MASK32)
       high 64 = t1 + ( (low_low_lo >> 32) )   -- need to add the upper 32 bits of low_low_lo into high part
    */
    uint64_t low64 = (t0_low32_lo << 32) | (low_low_lo & MASK32);
    uint64_t high64 = t1 + (low_low_lo >> 32);

    *lo = low64;
    *hi = high64;
}



void multiply_u64(uint64_t a, uint64_t b, uint64_t result128[2])
{
#if defined(__SIZEOF_INT128__) || defined(__GNUC__)
    uint64_t hi, lo;
    multiply_u64_native(a, b, &hi, &lo);
    result128[0] = lo;
    result128[1] = hi;
#else
    uint64_t hi, lo;
    multiply_u64_generic(a, b, &hi, &lo);
    result128[0] = lo;
    result128[1] = hi;
#endif
}




In [ ]:
%%writefile multiply-modulus.cu


///////


        /**
        Returns input mod modulus. This is not standard Barrett reduction.
        Correctness: modulus must be at most 63-bit.
        @param[in] input Should be at most 128-bit.
        */
        std::uint64_t barrett_reduce_128(const T *input, const Modulus &modulus)
        {
#ifdef SEAL_DEBUG
            if (!input)
            {
                throw std::invalid_argument("input");
            }
            if (modulus.is_zero())
            {
                throw std::invalid_argument("modulus");
            }
#endif
            // Reduces input using base 2^64 Barrett reduction
            // input allocation size must be 128 bits

            unsigned long long tmp1, tmp2[2], tmp3, carry;
            const std::uint64_t *const_ratio = modulus.const_ratio().data();

            // Multiply input and const_ratio
            // Round 1
            multiply_uint64_hw64(input[0], const_ratio[0], &carry);

            multiply_uint64(input[0], const_ratio[1], tmp2);
            tmp3 = tmp2[1] + add_uint64(tmp2[0], carry, &tmp1);

            // Round 2
            multiply_uint64(input[1], const_ratio[0], tmp2);
            carry = tmp2[1] + add_uint64(tmp1, tmp2[0], &tmp1);

            // This is all we care about
            tmp1 = input[1] * const_ratio[1] + tmp3 + carry;

            // Barrett subtraction
            tmp3 = input[0] - tmp1 * modulus.value();

            // One more subtraction is enough
            return SEAL_COND_SELECT(tmp3 >= modulus.value(), tmp3 - modulus.value(), tmp3);
        }

/*
SEAL code for modular multiplication of 2 64-bit numbers -
Multiplies two 64-bit integers → result is 128-bit.
Stores the 128-bit result in z[2].
Then performs Barrett reduction to reduce the 128-bit number modulo modulus.
*/
std::uint64_t multiply_uint_mod(
            std::uint64_t operand1, std::uint64_t operand2, const Modulus &modulus)
{
    unsigned long long z[2];    // uint64_t is same as unsigned long long
    multiply_uint64(operand1, operand2, z);
    return barrett_reduce_128(z, modulus);
}



In [ ]:
%%writefile miller-rabin-test.cu

/*
        Prepare for Miller-Rabin test -
        Requires writing  - value - 1 = 2^r * d , where d is odd and r is number of times 2 divides value - 1

        Choose a base a -
        First round: a = 2 (deterministic)
        Other rounds: random a from [3, value-1]

            Part 4 — Compute x=aᵈ mod value
            Then,
            do {
                x = multiply_uint_mod(x, x, modulus);
                count++;
            } while (x != value - 1 && count < r - 1);
            If we never hit value - 1, then value is definitely composite

        If all rounds succeed, then value is probably prime

        Miller-Rabin is very fast;for 64-bit numbers,a fre rounds guarantee primality
        SEAL-modulii are 50-60 bit, perfect for Miller-Rabin
*/

bool is_prime(uint64_t value, size_t num_rounds)
        {
            // First check the simplest cases.
            if (value < 2)
            {
                return false;
            }
            if (2 == value)
            {
                return true;
            }
            if (0 == (value & 0x1))
            {
                return false;
            }
            if (3 == value)
            {
                return true;
            }
            if (0 == (value % 3))
            {
                return false;
            }
            if (5 == value)
            {
                return true;
            }
            if (0 == (value % 5))
            {
                return false;
            }
            if (7 == value)
            {
                return true;
            }
            if (0 == (value % 7))
            {
                return false;
            }
            if (11 == value)
            {
                return true;
            }
            if (0 == (value % 11))
            {
                return false;
            }
            if (13 == value)
            {
                return true;
            }
            if (0 == (value % 13))
            {
                return false;
            }

            // Second, Miller-Rabin test.
            // Find r and odd d that satisfy value = 2^r * d + 1.
            uint64_t d = value - 1;
            uint64_t r = 0;
            while (0 == (d & 0x1))
            {
                d >>= 1;
                r++;
            }
            if (r == 0)
            {
                return false;
            }

            // 1) Pick a = 2, check a^(value - 1).
            // 2) Pick a randomly from [3, value - 1], check a^(value - 1).
            // 3) Repeat 2) for another num_rounds - 2 times.
            random_device rand;
            uniform_int_distribution<unsigned long long> dist(3, value - 1);
            for (size_t i = 0; i < num_rounds; i++)
            {
                uint64_t a = i ? dist(rand) : 2;
                uint64_t x = exponentiate_uint_mod(a, d, modulus);
                if (x == 1 || x == value - 1)
                {
                    continue;
                }
                uint64_t count = 0;
                do
                {
                    x = multiply_uint_mod(x, x, modulus);
                    count++;
                } while (x != value - 1 && count < r - 1);
                if (x != value - 1)
                {
                    return false;
                }
            }
            return true;
        }

int main()
{

}

In [ ]:
%%writefile miller-rabin-test.cu
/*
  miller-rabin-test.cu
  CUDA-parallel Miller-Rabin primality tester.

  Usage:
    nvcc -O3 miller-rabin-test.cu -o miller-rabin-test
    ./miller-rabin-test

  This file:
  - Implements modular multiplication and exponentiation using 128-bit intermediate.
  - Provides a device Miller-Rabin test that uses a per-test list of bases
    (host-prepared). The kernel runs one test per CUDA thread.
  - The host prepares random bases for each test (first round uses base 2
    deterministically as in the user's description).
*/

#include <cstdio>
#include <cstdint>
#include <cstdlib>
#include <vector>
#include <random>
#include <iostream>

#define CHECK_CUDA(call) do {                                  \
    cudaError_t err = (call);                                  \
    if (err != cudaSuccess) {                                  \
        fprintf(stderr, "CUDA error %s:%d: %s\n",              \
                __FILE__, __LINE__, cudaGetErrorString(err));  \
        exit(EXIT_FAILURE);                                    \
    }                                                          \
} while(0)


// 128-bit multiplication modulo using compiler __uint128_t.
// Marked __host__ __device__ so it can run on both CPU and GPU
static __inline__ __host__ __device__ uint64_t multiply_uint_mod(uint64_t a, uint64_t b, uint64_t mod)
{
    // Use 128-bit intermediate to avoid overflow for 64-bit inputs.
    // This is portable on NVCC/gcc/clang in modern toolchains.
    __uint128_t prod = ( __uint128_t ) a * ( __uint128_t ) b;
    return (uint64_t)(prod % mod);
}

// Modular exponentiation: compute base^exp % mod
static __inline__ __host__ __device__ uint64_t exponentiate_uint_mod(uint64_t base, uint64_t exp, uint64_t mod)
{
    uint64_t result = 1 % mod;
    uint64_t cur = base % mod;
    while (exp > 0)
    {
        if (exp & 1)
            result = multiply_uint_mod(result, cur, mod);
        cur = multiply_uint_mod(cur, cur, mod);
        exp >>= 1;
    }
    return result;
}

/*
  Device Miller-Rabin for a single value.
  - bases points to num_rounds bases for this value (first base should be 2 if you want deterministic first round).
  - Returns true => probably prime, false => composite.
*/
__device__ bool miller_rabin_single(uint64_t value, const uint64_t* bases, size_t num_rounds)
{
    // small checks (same as user's pre-checks)
    if (value < 2) return false;
    if (value == 2) return true;
    if ((value & 1ULL) == 0) return false;
    // check small primes quickly
    const uint64_t small_primes[] = {3,5,7,11,13};
    for (int i = 0; i < 5; ++i) {
        uint64_t p = small_primes[i];
        if (value == p) return true;
        if (value % p == 0) return false;
    }

    // write value - 1 = 2^r * d with d odd
    uint64_t d = value - 1;
    uint64_t r = 0;
    while ((d & 1ULL) == 0) {
        d >>= 1;
        ++r;
    }
    if (r == 0) return false; // defensive; though for odd value > 1, r >= 1

    for (size_t round = 0; round < num_rounds; ++round)
    {
        uint64_t a = bases[round];
        if (a % value == 0) { // base is multiple of value -> treat as trivial pass
            continue;
        }
        uint64_t x = exponentiate_uint_mod(a, d, value);
        if (x == 1 || x == value - 1) continue;
        uint64_t cnt = 0;
        bool hit = false;
        while (cnt < r - 1)
        {
            x = multiply_uint_mod(x, x, value);
            ++cnt;
            if (x == value - 1) { hit = true; break; }
        }
        if (!hit) return false;
    }
    return true;
}

/*
 Kernel:
 - values: array of input values length n
 - results: array of bools length n (0 or 1) to be filled
 - bases_all: contiguous array of uint64_t of length n * num_rounds.
    For value i, bases are at &bases_all[i * num_rounds]
*/
__global__ void miller_rabin_kernel(const uint64_t* values, uint8_t* results, const uint64_t* bases_all, size_t n, size_t num_rounds)
{
    size_t idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx >= n) return;
    const uint64_t v = values[idx];
    const uint64_t* my_bases = bases_all + idx * num_rounds;
    bool isprime = miller_rabin_single(v, my_bases, num_rounds);
    results[idx] = isprime ? 1 : 0;
}


/* Host helper: prepare bases for each value.
   - For each value i, first base is 2 (deterministic).
   - For subsequent rounds, pick uniformly random a in [3, value-1].
   - If value <= 3 we will just fill with 2s (safe).

  Called like: prepare_bases_for_all(host_values, host_bases_all, num_rounds);
*/
void prepare_bases_for_all(const std::vector<uint64_t>& values, std::vector<uint64_t>& bases_all, size_t num_rounds)
{
    std::random_device rd;
    std::mt19937_64 gen(rd());
    size_t n = values.size();
    bases_all.resize(n * num_rounds);
    for (size_t i = 0; i < n; ++i)
    {
        uint64_t v = values[i];
        uint64_t* out = bases_all.data() + i * num_rounds;
        // first round: a = 2
        out[0] = 2;
        if (num_rounds <= 1) continue;
        if (v <= 4) {
            // small values: use trivial bases
            for (size_t r = 1; r < num_rounds; ++r) out[r] = 2;
            continue;
        }
        std::uniform_int_distribution<uint64_t> dist(3, v - 1);
        for (size_t r = 1; r < num_rounds; ++r) {
            out[r] = dist(gen);
        }
    }
}


int main(int argc, char** argv)
{
    // Example usage: test a list of numbers on GPU.
    // You can modify this list to try more numbers.
    std::vector<uint64_t> host_values = {
        2ULL, 3ULL, 4ULL, 5ULL, 17ULL, 19ULL, 561ULL, 1105ULL, // 561 and 1105 are Carmichael (composite) numbers
        1000003ULL, 10000019ULL,
        2305843009213693951ULL // a large Mersenne prime candidate (M61) - fits in 64-bit
    };

    size_t n = host_values.size();
    size_t num_rounds = 8; // 8 rounds (user-specified parameter); for 64-bit small set of bases suffice, but we follow user's design

    // prepare bases on host
    std::vector<uint64_t> host_bases_all;
    prepare_bases_for_all(host_values, host_bases_all, num_rounds);

    // allocate device memory
    uint64_t* d_values = nullptr;
    uint64_t* d_bases = nullptr;
    uint8_t* d_results = nullptr;

    cudaMalloc((void**)&d_values, n * sizeof(uint64_t));
    cudaMalloc((void**)&d_bases, n * num_rounds * sizeof(uint64_t));
    cudaMalloc((void**)&d_results, n * sizeof(uint8_t));

    cudaMemcpy(d_values, host_values.data(), n * sizeof(uint64_t), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bases, host_bases_all.data(), n * num_rounds * sizeof(uint64_t), cudaMemcpyHostToDevice);

    // launch kernel
    int threads = 128;
    int blocks = (int)((n + threads - 1) / threads);
    miller_rabin_kernel<<<blocks, threads>>>(d_values, d_results, d_bases, n, num_rounds);
    cudaDeviceSynchronize();

    // copy results back
    std::vector<uint8_t> host_results(n);
    cudaMemcpy(host_results.data(), d_results, n * sizeof(uint8_t), cudaMemcpyDeviceToHost);

    // print
    for (size_t i = 0; i < n; ++i) {
        uint64_t v = host_values[i];
        bool isprime = !!host_results[i];
        std::cout << v << " -> " << (isprime ? "probably prime" : "composite") << "\n";
    }

    // cleanup
    cudaFree(d_values);
    cudaFree(d_bases);
    cudaFree(d_results);

    return 0;
}


1) poll and ioctl take most of time
2) cudaMallocManaged takes 210 ms - most amount of time - vs cudaLaunch
3) Actual GPU kernel execution takes 2.8 ms
4) 8.3 MB is size taken up by the cudaMallocManaged arrays to be used in device.

In [ ]:
# Nsight profiling
!nsys profile --stats=true --force-overwrite=true -o sys_report ./vector_add


A[100]=100.000000, B[100]=200.000000, C[100]=300.000000
Generating '/tmp/nsys-report-de3c.qdstrm'
[1/8] [========================100%] sys_report.nsys-rep
[2/8] [========================100%] sys_report.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/sys_report.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)     Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  -----------  --------  -----------  ------------  ----------------------
     76.8      530,982,116         18  29,499,006.4  6,578,484.0     1,265  290,044,475  69,398,018.7  poll                  
     22.6      156,246,343        549     284,601.7     13,656.0       506  104,152,026   4,495,538.3  ioctl                 
      0.3        1,893,471         31      61,079.7      9,868.0     8,065    1,216,600     215,456.5  mmap64           

Memory is loaded in GPU as Global DRAM -> L2 cache -> L1 cache -> Registers (per thread)
This loading is done in 128-byte chunks.
Say you want to do in GPU - A[i]=threadIdx.x; (all nearby indices of array would be accessed and this is coalesced access and such accesses are fast for texture memory  - texture memory is large read only global memory that supports this nearby access well).

Constant memory is much smaller (64 KB) and optimized for say, when multiple threads want to access same memory.